### <b>Table of Content</b>

0. Import functions

1. Download ZIP file from Google Drive and unzip in into local drive

2. Load image files and convert them into arrays

### <b>0. Import functions</b>

In [1]:
from utils.load import check_file_downloaded, extract_zip_file, load_images

import os

### <b>1. Download ZIP file from Google Drive and unzip in into local drive</b>

In [2]:
# Details of the source file in G Drive
file_id = "1KDQBTbo5deKGCdVV_xIujscn5ImxW4dm"
file_url = f"https://drive.google.com/file/d/{file_id}"
zip_file_name = "images.zip"

# Details of local directories
root_path = os.getcwd()
download_path = root_path + "\\" + "data"
zip_file_path = download_path + "\\" + zip_file_name

Download the source file from G Drive if the file does not exist.

In [3]:
os.chdir(download_path)
file_exists = os.path.exists(zip_file_name)
if file_exists:
    print(f"File {zip_file_name} already exists in {download_path}.")
else:
    print("Downloading file from Google Drive.")
    print("This could take a few minutes.")
    !gdown 1KDQBTbo5deKGCdVV_xIujscn5ImxW4dm

File images.zip already exists in c:\Users\Admin\Documents\GitHub\Apziva\lnaNWaYIRf6JhvHJ\data.


Check if the downloading was successful.

In [4]:
check_file_downloaded(file_name=zip_file_name, default_path=root_path, download_path=download_path)

File images.zip exist in c:\Users\Admin\Documents\GitHub\Apziva\lnaNWaYIRf6JhvHJ\data!


Extract the zip file.

In [5]:
extract_zip_file(zip_file_path, download_path, zip_file_name)

images.zip already extracted in c:\Users\Admin\Documents\GitHub\Apziva\lnaNWaYIRf6JhvHJ\data.


### <b>2. Load image files and convert them into arrays</b>

Load images as is for efficiency and memory saving.

In [6]:
array_dict = load_images(download_path, as_array=False)

Loading files in c:\Users\Admin\Documents\GitHub\Apziva\lnaNWaYIRf6JhvHJ\data\images
Loading files in c:\Users\Admin\Documents\GitHub\Apziva\lnaNWaYIRf6JhvHJ\data\images\testing
Loading files in c:\Users\Admin\Documents\GitHub\Apziva\lnaNWaYIRf6JhvHJ\data\images\testing\flip


100%|██████████| 290/290 [00:00<00:00, 387.54it/s]


Loading files in c:\Users\Admin\Documents\GitHub\Apziva\lnaNWaYIRf6JhvHJ\data\images\testing\notflip


100%|██████████| 307/307 [00:00<00:00, 553.77it/s]


Loading files in c:\Users\Admin\Documents\GitHub\Apziva\lnaNWaYIRf6JhvHJ\data\images\training
Loading files in c:\Users\Admin\Documents\GitHub\Apziva\lnaNWaYIRf6JhvHJ\data\images\training\flip


100%|██████████| 1162/1162 [00:01<00:00, 645.53it/s]


Loading files in c:\Users\Admin\Documents\GitHub\Apziva\lnaNWaYIRf6JhvHJ\data\images\training\notflip


100%|██████████| 1230/1230 [00:01<00:00, 615.26it/s]


In [7]:
len(array_dict["testing"]["flip"].keys())

290

In [ ]:
from numpy import asarray

# image = Image.open(jpg_file_path) <= values in the dict
image_array = asarray(image)
# Image.fromarray(image_array) # To revert back from array to image

# file_name = VideoID_FrameNumber

Convert each image into an array, and add VideoID, FrameNumber, and filp_or_not (boolean) for each array/image.
=> How to add these into a 3d array?

=> How to train a model using 3d arrays? Use neural networks / tensorflow?